<a href="https://colab.research.google.com/github/Aryanupadhyay23/Deep-Learning-/blob/main/Building_Model_Architectures_Alexnet_Lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub
path = kagglehub.dataset_download("princelv84/dogsvscats")

Using Colab cache for faster access to the 'dogsvscats' dataset.


In [ ]:
path

'/kaggle/input/dogsvscats'

In [ ]:
import os
os.listdir(path)

['test', 'train']

In [ ]:
os.listdir(f"{path}/train")

['dogs', 'cats']

In [ ]:
"""
import shutil
import os

src = "/kaggle/input/dogsvscats"
dst = "/content/dogs-vs-cats"

# Remove destination if it already exists (avoids errors)
if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(src, dst)
"""

'/content/dogs-vs-cats'

In [ ]:
# !pip install tensorflow numpy matplotlib keras

In [ ]:
# !pip install opencv-python

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random
import os

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import random
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
from keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
# Placeholder for the directory path
cat_directory_path = '/content/dogs-vs-cats/train/cats'
dog_directory_path = '/content/dogs-vs-cats/train/dogs'

In [ ]:
# Set path to the dataset
data_dir = '/content/dogs-vs-cats/train'

# Hyperparameters
img_width, img_height = 224, 224  # AlexNet input size
batch_size = 32
epochs = 20
validation_split = 0.2  # 20% of data for validation

In [ ]:
# Load datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=123,
    image_size=(img_width, img_height),
    batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Using 16000 files for training.


In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=123,
    image_size=(img_width, img_height),
    batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Using 4000 files for validation.


In [ ]:
# Data Augmentation layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

In [ ]:
# Apply data augmentation only on the training dataset
train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)
)

# Prefetch the datasets for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)


## Building Alexnet Model

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# AlexNet model
alexnet_model = Sequential([
    Input(shape=(224, 224, 3)),

    Conv2D(96, (11, 11), strides=(4, 4), activation='relu'),
    MaxPooling2D((3, 3), strides=(2, 2)),

    Conv2D(256, (5, 5), padding='same', activation='relu'),
    MaxPooling2D((3, 3), strides=(2, 2)),

    Conv2D(384, (3, 3), padding='same', activation='relu'),
    Conv2D(384, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((3, 3), strides=(2, 2)),

    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
alexnet_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
alexnet_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 54, 54, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 26, 26, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 26, 26, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    26,218,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 140,253,317 (535.02 MB)

 Trainable params: 46,751,105 (178.34 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 93,502,212 (356.68 MB)

In [ ]:
"""
history = alexnet_model.fit(train_dataset, epochs=5, validation_data = validation_dataset)
"""

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 215s 409ms/step - accuracy: 0.5012 - loss: 9.7574 - val_accuracy: 0.4918 - val_loss: 0.6939
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 195s 390ms/step - accuracy: 0.4931 - loss: 0.6942 - val_accuracy: 0.4918 - val_loss: 0.6940
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 190s 380ms/step - accuracy: 0.4957 - loss: 0.6937 - val_accuracy: 0.4915 - val_loss: 0.6938
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 191s 382ms/step - accuracy: 0.4960 - loss: 0.6940 - val_accuracy: 0.4915 - val_loss: 0.6940
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 194s 389ms/step - accuracy: 0.5003 - loss: 0.6935 - val_accuracy: 0.4915 - val_loss: 0.6953


In [ ]:
# Evaluate the model on validation data
val_loss, val_accuracy = alexnet_model.evaluate(validation_dataset)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - accuracy: 0.4950 - loss: 0.6949
Validation Accuracy: 49.15%


In [ ]:
# Save the model
alexnet_model.save("saved_model_5_epochs.keras")

In [ ]:
# Save the model weights
alexnet_model.save_weights("model_5_epochs.weights.h5")

In [ ]:
# Load the model
from tensorflow.keras.models import load_model

loaded_model = load_model("saved_model_5_epochs.keras")

# Now you can use the loaded model for inference or further training

In [ ]:
loaded_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 54, 54, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 26, 26, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 26, 26, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 12, 12, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │    26,218,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 140,253,317 (535.02 MB)

 Trainable params: 46,751,105 (178.34 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 93,502,212 (356.68 MB)

In [ ]:
"""
history = alexnet_model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=2
)
"""

Epoch 1/2
500/500 ━━━━━━━━━━━━━━━━━━━━ 193s 385ms/step - accuracy: 0.5008 - loss: 0.6946 - val_accuracy: 0.4925 - val_loss: 0.6958
Epoch 2/2
500/500 ━━━━━━━━━━━━━━━━━━━━ 194s 388ms/step - accuracy: 0.4850 - loss: 0.6940 - val_accuracy: 0.4915 - val_loss: 0.6940


In [ ]:
history_dict = history.history

# Building Lenet Model

In [ ]:
# LeNet model
lenet_model = Sequential([
    Conv2D(32, (5, 5), activation='tanh', input_shape=(32, 32, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (5, 5), activation='tanh'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='tanh'),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
lenet_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
lenet_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 28, 28, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 10, 10, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 257,153 (1004.50 KB)

 Trainable params: 257,153 (1004.50 KB)

 Non-trainable params: 0 (0.00 B)